In [ ]:
!pip install diffusers transformers accelerate scipy safetensors streamlit py-localtunnel

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 46.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.6 MB/s eta 0:00:00
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 7.0.0
    Uninstalling importlib-metadata-7.0.0:
      Successfully uninstalled importlib-metadata-7.0.0


In [ ]:
%%writefile app.py
from diffusers import StableDiffusionPipeline,StableDiffusionXLPipeline
import matplotlib.pyplot as plt
import torch
pipe = StableDiffusionXLPipeline.from_pretrained("segmind/SSD-1B", torch_dtype=torch.float16, use_safetensors=True, variant="fp16")
pipe = pipe.to("cuda")

Writing app.py


In [ ]:
!python app.py

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.
0it [00:00, ?it/s]
2024-01-08 12:32:58.485638: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-08 12:32:58.485701: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-08 12:32:58.487868: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-08 12:33:00.827279: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find

In [ ]:
%%writefile main.py
from diffusers import StableDiffusionPipeline,StableDiffusionXLPipeline
import matplotlib.pyplot as plt
import torch
import streamlit as st
from app import pipe
style_list = [
    {
        "name": "(No style)",
        "prompt": "{prompt}",
        "negative_prompt": "",
    },
    {
        "name": "Cinematic",
        "prompt": "cinematic still {prompt} . emotional, harmonious, vignette, highly detailed, high budget, bokeh, cinemascope, moody, epic, gorgeous, film grain, grainy",
        "negative_prompt": "anime, cartoon, graphic, text, painting, crayon, graphite, abstract, glitch, deformed, mutated, ugly, disfigured",
    },
    {
        "name": "Photographic",
        "prompt": "cinematic photo {prompt} . 35mm photograph, film, bokeh, professional, 4k, highly detailed",
        "negative_prompt": "drawing, painting, crayon, sketch, graphite, impressionist, noisy, blurry, soft, deformed, ugly",
    },
    {
        "name": "Anime",
        "prompt": "anime artwork {prompt} . anime style, key visual, vibrant, studio anime,  highly detailed",
        "negative_prompt": "photo, deformed, black and white, realism, disfigured, low contrast",
    },
    {
        "name": "Manga",
        "prompt": "manga style {prompt} . vibrant, high-energy, detailed, iconic, Japanese comic style",
        "negative_prompt": "ugly, deformed, noisy, blurry, low contrast, realism, photorealistic, Western comic style",
    },
    {
        "name": "Digital Art",
        "prompt": "concept art {prompt} . digital artwork, illustrative, painterly, matte painting, highly detailed",
        "negative_prompt": "photo, photorealistic, realism, ugly",
    },
    {
        "name": "Pixel art",
        "prompt": "pixel-art {prompt} . low-res, blocky, pixel art style, 8-bit graphics",
        "negative_prompt": "sloppy, messy, blurry, noisy, highly detailed, ultra textured, photo, realistic",
    },
    {
        "name": "Fantasy art",
        "prompt": "ethereal fantasy concept art of  {prompt} . magnificent, celestial, ethereal, painterly, epic, majestic, magical, fantasy art, cover art, dreamy",
        "negative_prompt": "photographic, realistic, realism, 35mm film, dslr, cropped, frame, text, deformed, glitch, noise, noisy, off-center, deformed, cross-eyed, closed eyes, bad anatomy, ugly, disfigured, sloppy, duplicate, mutated, black and white",
    },
    {
        "name": "Neonpunk",
        "prompt": "neonpunk style {prompt} . cyberpunk, vaporwave, neon, vibes, vibrant, stunningly beautiful, crisp, detailed, sleek, ultramodern, magenta highlights, dark purple shadows, high contrast, cinematic, ultra detailed, intricate, professional",
        "negative_prompt": "painting, drawing, illustration, glitch, deformed, mutated, cross-eyed, ugly, disfigured",
    },
    {
        "name": "3D Model",
        "prompt": "professional 3d model {prompt} . octane render, highly detailed, volumetric, dramatic lighting",
        "negative_prompt": "ugly, deformed, noisy, low poly, blurry, painting",
    },
]

st.set_page_config(
    page_title="Dreamy Wallpaper Generator",
    layout="wide",  # Ensure maximum width for image display
)

# Add title and introduction

st.markdown(
    "<h1 style='text-align: center;'>Create Stunning Personalized Wallpapers</h1>",
    unsafe_allow_html=True,
)

# Create input elements with clear labels
prompt = st.text_input(
    "Describe your dream wallpaper:",
    placeholder="e.g., A vibrant cyberpunk cityscape with neon lights at night",
)

st.subheader("Select Image Styles")
selected_styles = st.selectbox(
    "Choose a style:",
    [style["name"] for style in style_list]
)

st.subheader("Choose Resolution")
resolution = st.selectbox(
    "Select desired resolution:",
    [
        "1152x896",
        "896x1152",
        "1216x832",
        "1024x1024",
        "832x1216",
        "1344x768",
        "768x1344",
        "1536x640",
        "640x1536",
    ],
)
def gen_image(prompt, size, selected_styles):
    try:
        width, height = map(int, size.split("x"))
        selected_style_prompts = [
            style["prompt"].format(prompt=prompt)
            for style in style_list if style["name"] in selected_styles
        ]
        refined_prompt = ", ".join(selected_style_prompts)  # Combine prompts

        selected_negative_prompts = [
            style["negative_prompt"]
            for style in style_list if style["name"] in selected_styles
        ]
        negative_prompt = ", ".join(selected_negative_prompts)  # Combine negative prompts

        image = pipe(refined_prompt,
                      negative_prompt=negative_prompt,
                      width=width, height=height,
                      image_style=selected_styles).images[0]
        return image
    except (ValueError, RuntimeError) as e:
        st.error(f"Error generating image: {e}")
        return None
download_image = None
# Add a button for image generation (you'll add functionality later)
if st.button("Generate Wallpaper"):
    image = gen_image(prompt, resolution, selected_styles)
    if image:  # Check if image generation succeeded
        st.subheader("Your Dream Wallpaper")
        st.image(image,use_column_width=False)




Writing main.py


In [ ]:
!streamlit run main.py & npx localtunnel --port 8501

[..................] / rollbackFailedOptional: verb npm-session 8ef7b2fd006c6e7


  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://34.148.157.196:8501

npx: installed 22 in 5.277s
your url is: https://poor-webs-shake.loca.lt
2024-01-08 12:35:20.626160: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-08 12:35:20.626219: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-08 12:35:20.627825: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-08 12:35:21.954819: W tensorflow/compiler/tf2tensorrt/utils/py_